#### Install dependencies

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# New OS parameter to avoid warnings.  
# This will not have a material impact on your code, but prevents warnings from appearing related to new LangChain features.
import os
os.environ['USER_AGENT'] = 'RAGUserAgent'

In [3]:
import openai
import bs4
import chromadb

from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain import hub
from langchain_core.documents.base import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain_experimental.text_splitter import SemanticChunker
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever


from modules import utils

from typing import List

In [4]:
envs = utils.load_env_file("./../secrets/env")

In [5]:
pdf_path = "vks.pdf"

# Indexing
## Web loading and crawling

In [6]:
# Load the PDF and extract text
pdf_reader = PdfReader(pdf_path)
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

In [7]:
print(text)

1ThuyVT2
2VKS
VKS  VNGCloud Kubernetes Service) is a managed service on VNGCloud that helps
you simplify the deployment and management of container-based applications.
Kubernetes is an open-source platform developed by Google, widely used for
managing and deploying containerized applications in distributed environments.
VKS Demo Video - Gi ả i Pháp Qu ả n Lý Kubernetes Toàn Di ệ n C ủ a VN VKS Demo Video - Gi ả i Pháp Qu ả n Lý Kubernetes Toàn Di ệ n C ủ a VN ……
3What is VKS?
VKS  VNGCloud Kubernetes Service) is a managed service on VNGCloud that
simplifies the deployment and management of container-based applications.
Kubernetes, an open-source platform developed by Google, is widely used to
manage and deploy containerized applications in distributed environments.
•Fully Managed control plane: VKS will free you from the burden of managing
the Kubernetes Control Plane, allowing you to focus on developing applications.
Highlights of VKS
4•Support for the latest Kubernetes versions: VKS 

## Splitting

In [8]:
# Split
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=200
)
splits = character_splitter.split_text(text)

In [9]:
dense_documents = [Document(page_content=text, metadata={"id": str(i), "source": "dense"}) for i, text in enumerate(splits)]
sparse_documents = [Document(page_content=text, metadata={"id": str(i), "source": "sparse"}) for i, text in enumerate(splits)]

## Embedding

In [10]:
embedding_model = "llama3.1:8b"
base_url = envs["OLLAMA_HOST"]  # for example, "http://localhost:11434"

embedding_function = OllamaEmbeddings(base_url=base_url, model=embedding_model)

In [11]:
collection_name = "vks"

In [12]:
chroma_client = chromadb.Client()
vectorstore = Chroma.from_documents(
    documents=dense_documents,
    embedding=embedding_function,
    collection_name=collection_name,
    client=chroma_client
)

In [13]:
# Create dense retriever
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
# Create sparse retriever
sparse_retriever = BM25Retriever.from_documents(sparse_documents, k=10)
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[dense_retriever, sparse_retriever], weights=[0.5, 0.5], c=0)

## Retrieval

In [14]:
# Prompt - ignore LangSmith warning, you will not need langsmith for this coding exercise
prompt = hub.pull("jclemens24/rag-prompt")

/mnt/kalista/git-cuongpiger/unlocking-data-with-generative-ai-and-rag/.venv/lib/python3.10/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [15]:
# Relevance check prompt
relevance_prompt_template = PromptTemplate.from_template(
    """
    You are an expert evaluator tasked with determining the relevance of a given context to a specific question.
    Question: "{question}"
    Context: "{retrieved_context}"
    
    Assign a relevance score from 1 to 5 based on the following guidelines:
    - 1: Not at all relevant
    - 2: Slightly relevant
    - 3: Moderately relevant
    - 4: Very relevant
    - 5: Highly relevant
    Provide ONLY the numeric score as your response without any additional text.

    Relevance Score:
    """
)

In [16]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [17]:
def extract_score(llm_output):
    try:
        score = float(llm_output.strip())
        return score
    except ValueError:
        return 0

# Chain it all together with LangChain
def conditional_answer(x):
    relevance_score = extract_score(x['relevance_score'])
    if relevance_score < 4:
        return "I don't know."
    else:
        return x['answer']

In [18]:
llm = ChatOllama(base_url=base_url, 
                 model=embedding_model,
                 temperature=0)

In [19]:
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | RunnableParallel(
        {"relevance_score": (
            RunnablePassthrough()
            | (lambda x: relevance_prompt_template.format(question=x['question'], retrieved_context=x['context']))
            | llm
            | StrOutputParser()
        ), "answer": (
            RunnablePassthrough()
            | prompt
            | llm
            | StrOutputParser()
        )}
    )
    | RunnablePassthrough().assign(final_answer=conditional_answer)
)

In [20]:
rag_chain_with_source = RunnableParallel(
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [21]:
# User Query
user_query = "What is VKS"

result = rag_chain_with_source.invoke(user_query)
retrieved_docs = result['context']

print(f"Original Question: {user_query}\n")
print(f"Relevance Score: {result['answer']['relevance_score']}\n")
print(f"Final Answer:\n{result['answer']['final_answer']}\n\n")
print("Retrieved Documents:")
for i, doc in enumerate(retrieved_docs, start=1):
    print(f"Document {i}: Document ID: {doc.metadata['id']} source: {doc.metadata['source']}")
    print(f"Content:\n{doc.page_content}\n")

Original Question: What is VKS

Relevance Score: 3

Final Answer:
I don't know.


Retrieved Documents:
Document 1: Document ID: 26 source: dense
Content:
Specifically, the Node Name will have additional information: Cluster Name ,
Node Group Name .
◦Delete User Builder on User's Worker Node.
◦Change SSH mechanism from Port 22 to Port 234.
If you encounter any problems with this official release, please contact VKS support
for assistance.
May 03, 2024
20The latest update for VKS is available, bringing many new features and
improvements to users. Here are the details about the update:
New feature:
•Supports Whitelist feature: VKS allows creating a Private Node Group with only
Private IP and also allows any IP to connect to the Cluster through the Whitelist
IP feature. For more details, please refer to Whitelist .
Improve:
•System optimization: Helps the system operate more smoothly and efficiently.
•Bug Fixes: Fixed some minor bugs to provide a better user experience.
If you encounter an